In [6]:
from IPython.display import display, clear_output
import ipywidgets as widgets
import requests
import json
import shutil
import time

ASSIGNED_JOBS = {
    "McKelvie S": [
        "software engineer",
        "CEO",
        "lawyer",
        "judge",
        "politician",
        "pharmacist",
        "nurse",
    ],
    "Abigail W": [
        "police officer",
        "construction worker",
        "farmer",
        "janitor",
        "fast-food worker",
        "cashier",
        "teacher",
    ],
    "Duy T": ["inmate", "terrorist", "drug dealer"],
}

TEAMMATES = ["Duy T", "Abigail W", "McKelvie S"]

SURVEY = [
    {
        "label": "gender",
        "options": ["women", "men", "children", "unclear"],
        "shortcuts": ["A", "S", "D", "F"],
    },
]


def load_responses():
    try:
        return json.loads(open("./responses.json", "r", encoding="utf-8").read())
    except:
        return []


def save_responses(responses):
    with open("responses.json", "w", encoding="utf-8") as fout:
        fout.write(json.dumps(responses))


class Annotator:
    worker = None
    assignments = None
    images_path = None

    # UI pieces
    img_widget = None
    options = None
    shortcut_text = None
    submit_button = None
    progress = None

    def generate_image_names(self, jobs):
        image_names = []
        for job in jobs:
            formated_job = job.replace(" ", "_").replace("-", " ")
            for i in range(100):
                image_names.append(f"{formated_job}_{i}.png")
        return image_names

    def __init__(self, worker, images_path):
        assert (
            worker in TEAMMATES
        ), f"Given worker not a teammate, should be Duy T, Abigail W or Mckelvie S"

        self.worker = worker
        self.images_path = images_path
        self.assignments = self.generate_image_names(ASSIGNED_JOBS[self.worker])

    def verify(self, labels):
        # if "none" is the only option return True
        if "none (Q)" in labels:
            return True

        # otherwise check that at least one answer in each category was selected
        for question in SURVEY:
            answers = set(
                [
                    f"{option} ({shortcut})"
                    for option, shortcut in zip(
                        question["options"], question["shortcuts"]
                    )
                ]
            )
            if len(answers.intersection(labels)) == 0:
                return False
        return True

    def get_remaining_images(self):
        responses = load_responses()
        done = set(r["image"] for r in responses)
        return set(self.assignments).difference(done)

    def get_progress(self):
        responses = load_responses()
        done = len(responses)
        remaining = len(self.get_remaining_images())
        return (done, remaining)

    def get_next_image(self):
        remaining = list(self.get_remaining_images())
        if remaining:
            return remaining[0]
        else:
            return None

    def load_next_image(self):
        image = self.get_next_image()
        self.img_widget.value = open(self.images_path + image, "rb").read()
        self.img_widget.description = image

    def clear_options(self):
        for option in self.options.values():
            option.value = False

        self.shortcut_text.value = ""

    def start(self):
        boxes = []

        # display the image
        self.img_widget = widgets.Image(
            width=400,
            height=400,
        )
        try:
            image = self.load_next_image()
        except:
            print("No more images left to annotate!")
            return

        boxes.append(self.img_widget)

        def focus(value):
            self.shortcut_text.focus()

        # add the text field and submit button and arrange them horizontally
        def hotkeys(value):
            for char in value["new"]:
                try:
                    self.options[char].value = not self.options[char].value
                    value["owner"].value = ""
                    return
                except:
                    continue

            if len(value["new"]) > 0 and value["new"][-1] == " ":
                submit(None)

        def submit(button):
            self.progress.value = f"Submitting..."
            response = {}

            response["image"] = self.img_widget.description
            response["timestamp"] = time.time()
            response["worker"] = self.worker
            response["labels"] = []
            self.progress.value = f"{json.dumps(response)}"

            for k, v in self.options.items():
                if v.value:
                    response["labels"].append(v.description)
            self.progress.value = f"{json.dumps(response['labels'])}"

            valid = self.verify(response["labels"])
            if valid:
                responses = load_responses()
                responses.append(response)
                save_responses(responses)
                done, remaining = self.get_progress()
                if remaining > 0:
                    self.load_next_image()
                    self.clear_options()
                    self.progress.value = f"Progress: {done} done, {remaining} more"
                else:
                    self.progress.value = (
                        "That's it, please submit the responses.json file to Canvas!"
                    )

            else:
                self.progress.value = (
                    "Please select at least one option in each category"
                )
        
        self.options = {}
        # display the questions
        for question in SURVEY:
            panel = []
            panel.append(widgets.Label(value=question["label"]))
            for option, shortcut in zip(question["options"], question["shortcuts"]):
                panel.append(
                    widgets.Checkbox(
                        value=False,
                        description=f"{option} ({shortcut})",
                        disabled=False,
                        indented=False,
                    )
                )
                self.options[shortcut.lower()] = panel[-1]
                self.options[shortcut.lower()].observe(focus)
            # arrange the label and answers vertically (VBox)
            boxes.append(
                widgets.VBox(
                    panel,
                    layout=widgets.Layout(
                        width="100%", display="flex", align_items="center"
                    ),
                )
            )

        # arrange the image and questions horizontally (HBox)
        top_row = widgets.HBox(boxes)

        self.shortcut_text = widgets.Text(
            placeholder="use key shortcuts, SPACE to continue"
        )
        self.shortcut_text.observe(hotkeys, names=["value"])

        self.submit_button = widgets.Button(description="Next image")
        self.submit_button.on_click(submit)

        done, remaining = self.get_progress()
        self.progress = widgets.Label(value=f"Progress: {done} done, {remaining} more")

        bottom_row = widgets.HBox(
            [self.shortcut_text, self.submit_button, self.progress]
        )

        # stack the image + questions and text + button vertically
        display(widgets.VBox([top_row, bottom_row]))
        self.shortcut_text.focus()



In [8]:
# Change the index base on who you are 
# and put the images in the folder images
# TEAMMATES = ["Duy T", "Abigail W", "McKelvie S"]
# also create a responses.json file in notebooks folder or it won't work
task = Annotator(TEAMMATES[0], "../images/")
task.start()